In [1]:
import pandas as pd
import os
import copyimport copy
import numpy as np
import re
from datetime import datetime

In [2]:
#Vehicle List by VIN and Model Type
col_list = ["VIN", "VEHICLE_TYPE", "MODEL_TYPE", "MODEL_CODE", "DEPT_PORT"]
iter_csv = pd.read_csv(r'\\hke.local\Glovis\Public\Power BI\MASTER LISTS\Vehicle List\vehicle_list_rolling_year.csv', iterator=True, chunksize=4000, usecols=col_list)
vl_pull = pd.concat(chunk.loc[(chunk['DEPT_PORT'] == "HU")] for chunk in iter_csv)
vl_mdl_cd = vl_pull

In [3]:
#HU ANT list
hu_ant = pd.read_excel(r'\\hke.local\Glovis\Dept\PI\HQ Projects\PP Optimization 2022\iVELES Available Manual Pull 012722.xlsx')

In [4]:
#HU ANT list
hu_lts_mapping = pd.read_excel(r'\\hke.local\Glovis\Dept\PI\HQ Projects\PP Optimization 2022\ITMS Code LTS Mapping Contract.xlsx')

In [19]:
pd.set_option('display.max_columns', None)

In [5]:
hu_lts_mapping

,Port,Customer,No,Rate Start Date,Rate End Date,WO Code,Description,Cost Flag,Vehicle Type,Model Type,Accry. Lbr. Time,PDI Time,Warranty Time,Body Shop Time,Fixed Rate,Mark Up Rate,Rate,Skilled YN,Use YN
0,HU,HMA,1,01/01/2022,12/31/2022,77,Load Capacity Labels - Installs only,No,R5 - Palisade,Palisade,0.025,0.0,0,0,0.0,0,0.72225,N,Yes
1,HU,HMA,2,01/01/2022,12/31/2022,75,Tow Hook Removal (2),No,R5 - Palisade,Palisade,0.120,0.0,0,0,0.0,0,3.98640,N,Yes
2,HU,HMA,3,01/01/2022,12/31/2022,74,Tow Hook Removal (1),No,R5 - Palisade,Palisade,0.060,0.0,0,0,0.0,0,1.99320,N,Yes
3,HU,HMA,4,01/01/2022,12/31/2022,69,XM Radio Brochure,No,R5 - Palisade,Palisade,0.005,0.0,0,0,0.0,0,0.14445,N,Yes
4,HU,HMA,5,01/01/2022,12/31/2022,68,XM Check at Final Inspection,No,R5 - Palisade,Palisade,0.050,0.0,0,0,0.0,0,1.44450,N,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3475,HU,KMA,2352,01/01/2021,12/31/2021,12,Collect Key Code,No,E3 - Stinger,Stinger,0.050,0.0,0,0,0.0,0,2.11700,N,Yes
3476,HU,KMA,2353,01/01/2021,12/31/2021,12,Collect Key Code,No,E2 - Stinger,Stinger,0.050,0.0,0,0,0.0,0,2.11700,N,Yes
3477,HU,KMA,2354,01/01/2021,12/31/2021,12,Collect Key Code,No,E1 - Stinger,Stinger,0.050,0.0,0,0,0.0,0,2.11700,N,Yes
3478,HU,KMA,2355,01/01/2021,12/31/2021,12,Collect Key Code,No,CM - Niro PHEV,Niro PHEV,0.050,0.0,0,0,0.0,0,2.11700,N,Yes


In [11]:
hu_lts_mapping['Vehicle Type'] = [cd[0:cd.find(' ')] for cd in hu_lts_mapping['Vehicle Type']]

In [6]:
hu_ant_vtype = pd.merge(hu_ant, vl_mdl_cd[['VEHICLE_TYPE']], left_on = hu_ant['VIN'], right_on = vl_mdl_cd['VIN'], how = 'left')

In [17]:
hu_ant_vtype

,key_0,No,Port,VIN,Status,Customer,VPS,Mani Aloc Date,Mani Aloc No.,Model Type,...,Location Update Date,Vessel,Vessel ETA,In Process Date,Arrival Date,Rail Car No.,XM ID,Inspector ID,Throw In ID,VEHICLE_TYPE
0,KM8KM4AE9NU052589,1,HU,KM8KM4AE9NU052589,[N]Normal Port Stock,HMA,[S2000]Stock,12/16/2021,2021121600002,[IQ5]IONIQ 5 (NE1),...,01/05/2022 09:44:48,MSC CRIS,12/05/2021,12/20/2021,12/08/2021,NaN,NaN,NaN,NaN,5IQ
1,KM8S6DA25NU033540,2,HU,KM8S6DA25NU033540,[N]Normal Port Stock,HMA,[S2000]Stock,12/22/2021,2022010500011,[SFP]SANTA FE PHEV (TMa),...,01/11/2022 09:41:43,LAKE GEN,01/05/2022,01/12/2022,01/08/2022,NaN,NaN,NaN,NaN,S0
2,KMHT36AH1NU013136,3,HU,KMHT36AH1NU013136,[N]Normal Port Stock,HMA,[S2000]Stock,12/22/2021,2021122200307,[VL1]VELOSTER (JS),...,01/11/2022 09:50:41,LAKE GEN,01/05/2022,01/12/2022,01/08/2022,NaN,NaN,NaN,NaN,T3
3,KNDJ23AU4N7821344,4,HU,KNDJ23AU4N7821344,[N]Normal Port Stock,KMA,[S2000]Stock,11/19/2021,2021122300745,[SO1]SOUL (SK3),...,01/20/2022 13:19:48,DIGNITY,01/15/2022,01/21/2022,01/18/2022,NaN,NaN,NaN,NaN,J2
4,KNDJ23AU6N7822270,5,HU,KNDJ23AU6N7822270,[N]Normal Port Stock,KMA,[S2000]Stock,11/23/2021,2021122300745,[SO1]SOUL (SK3),...,01/20/2022 13:19:48,DIGNITY,01/15/2022,01/21/2022,01/18/2022,NaN,NaN,NaN,NaN,J2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4333,KMHLM4AG3NU301944,4334,HU,KMHLM4AG3NU301944,[N]Normal Port Stock,HMA,[S2000]Stock,01/18/2022,2022011800341,[EL2]ELANTRA (CN7),...,01/21/2022 10:38:55,DREAM DI,01/20/2022,01/21/2022,01/20/2022,NaN,NaN,NaN,NaN,D7
4334,KMHLW4AKXNU002384,4335,HU,KMHLW4AKXNU002384,[N]Normal Port Stock,HMA,[S2000]Stock,01/18/2022,2022011800338,[EL2]ELANTRA (CN7),...,01/21/2022 10:38:55,DREAM DI,01/20/2022,01/21/2022,01/20/2022,NaN,NaN,NaN,NaN,D6
4335,KMHLM4AJ1NU020222,4336,HU,KMHLM4AJ1NU020222,[N]Normal Port Stock,HMA,[S2000]Stock,01/18/2022,2022011800338,[EH1]ELANTRA HEV (CN7),...,01/21/2022 10:38:56,DREAM DI,01/20/2022,01/21/2022,01/20/2022,NaN,NaN,NaN,NaN,D9
4336,KMHLM4AJ0NU020339,4337,HU,KMHLM4AJ0NU020339,[N]Normal Port Stock,HMA,[S2000]Stock,01/18/2022,2022011800338,[EH1]ELANTRA HEV (CN7),...,01/21/2022 10:38:56,DREAM DI,01/20/2022,01/21/2022,01/20/2022,NaN,NaN,NaN,NaN,D9


In [28]:
def average(lst):
    if len(lst) > 0:
        avg = sum(lst) / len(lst)
    else:
        avg = 0
    return avg

In [36]:
#LTS Mapping - direct lookup for one-to-one match b/w vehicle type and PIO code, average calculation otherwise
light_pio_hrs = []
heavy_pio_hrs = []

for i in range(0, len(hu_ant_vtype)):
    
    #If there is no PIO code, put empty list into PIO_code_iter. Else break up comma-delimited entry to list
    if pd.isnull(hu_ant_vtype['PIO Code'][i]):
        PIO_code_iter = []
    else:
        PIO_code_iter = hu_ant_vtype['PIO Code'][i].split(',')
    
    #Total light and heavy PIO hours
    light_pio_entry = 0
    heavy_pio_entry = 0
    
    #For codes in PIO_code_iter, calculate LTS for non-numeric codes only (PIO)
    for cd in PIO_code_iter:
        if not cd.isnumeric():
            vtype = hu_ant_vtype['VEHICLE_TYPE'][i]
            cust = hu_ant_vtype['Customer'][i]
            
            #Filter down LTS table by vehicle type, WO code, and customer
            hu_lts_restr = list(hu_lts_mapping.loc[(hu_lts_mapping['Vehicle Type'] == vtype) & (hu_lts_mapping['WO Code'] == cd) & (hu_lts_mapping['Customer'] == cust)]['Accry. Lbr. Time'])
            
            #If clean match, put LTS between 0.05 and 0.5 into light, and above 0.5 into heavy (<= 0.05 are throw-ins)
            if len(hu_lts_restr) == 1:
                if(hu_lts_restr[0] < 0.5 and hu_lts_restr[0] > 0.05):
                    light_pio_entry += hu_lts_restr[0]
                elif (hu_lts_restr[0] >= 0.5):
                    heavy_pio_entry += hu_lts_restr[0]
            
            #Else repeat but apply average for WO code and customer
            else:
                hu_lts_restr = list(hu_lts_mapping.loc[(hu_lts_mapping['WO Code'] == cd) & (hu_lts_mapping['Customer'] == cust)]['Accry. Lbr. Time'])
                
                if(average(hu_lts_restr) < 0.5 and average(hu_lts_restr) > 0.05):
                    light_pio_entry += average(hu_lts_restr)
                elif (average(hu_lts_restr) >= 0.5):
                    heavy_pio_entry += average(hu_lts_restr)
            
    light_pio_hrs.append(round(light_pio_entry, 2))
    heavy_pio_hrs.append(round(heavy_pio_entry, 2))        

In [37]:
hu_ant_vtype['Light PIO Hrs'] = light_pio_hrs
hu_ant_vtype['Heavy PIO Hrs'] = heavy_pio_hrs

In [38]:
hu_ant_vtype

,key_0,No,Port,VIN,Status,Customer,VPS,Mani Aloc Date,Mani Aloc No.,Model Type,Model Year,Model,Acce Grup Cd,Carrier,Dealer,Dest,Req Port Rels Date,Ship To Dealer,Ship.Dealer Type,Ship to City,Ship to State,Region,Ext. Color,Int. Color,Production Date,Received,Allocation,Allocation Date,Allocated,Sold Status,Instruction,Drop Ship,VPC In Date,VPC Out Date,Release Date,Shipment Date,Shipment No.,Load No.,Load Date,Rail Load Position,Trans. Type,De-commit Date,PUP No.,PUP Date,PUP Open Flag,Activity Code,PIO Code,Install Type,HOLD,HOLD Code,HOLD Comment,R/O No.,R/O Date,Start Due Date,End Due Date,Work Order Print Date,Received Aged,Allocated Days Aged,comment,Location,Location Update,Location Update Date,Vessel,Vessel ETA,In Process Date,Arrival Date,Rail Car No.,XM ID,Inspector ID,Throw In ID,VEHICLE_TYPE,Light PIO Hrs,Heavy PIO Hrs
0,KM8KM4AE9NU052589,1,HU,KM8KM4AE9NU052589,[N]Normal Port Stock,HMA,[S2000]Stock,12/16/2021,2021121600002,[IQ5]IONIQ 5 (NE1),2022,[50412REZ]MY22 IONIQ 5 (NE1),1,[MPG]United Road,[AZ043]EARNHARDT HYUNDAI NORTH SCOTTS,NaN,NaN,[AZ043]EARNHARDT HYUNDAI NORTH SCOTTS,Dealer Car,SCOTTSDALE,AZ,[MS],[SAW]Lunar White,[YGN]Gray,11/09/2021,12/09/2021,3667386,12/16/2021,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,MUFK02 NECFF1 NECN01 NECT01,"CF,CN,CT,FK",Throw-in PIO,No,NaN,No cmnt hold,NaN,NaN,NaN,NaN,01/21/2022 13:54:41,50,42,NaN,15-06-050-003,aflores,01/05/2022 09:44:48,MSC CRIS,12/05/2021,12/20/2021,12/08/2021,NaN,NaN,NaN,NaN,5IQ,0.10,0.0
1,KM8S6DA25NU033540,2,HU,KM8S6DA25NU033540,[N]Normal Port Stock,HMA,[S2000]Stock,12/22/2021,2022010500011,[SFP]SANTA FE PHEV (TMa),2022,[644D2APT]MY22 SANTA FE PHEV (TMa),1,"[EXTRM]Extreme Transportation, Inc.",[CA320]DUBLIN HYUNDAI,NaN,NaN,[CA320]DUBLIN HYUNDAI,Dealer Car,DUBLIN,CA,[WE][KMA/HMA] Western,[WW8]Quartz White,[NNB]Black,12/08/2021,01/11/2022,3680133,12/22/2021,Y,[S]Sold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,97 MUCFK5 TMCCN1 TMCCT2 TMCV01 TMHCF1,"C1,CF,CV",Light PIO,No,NaN,No cmnt hold,NaN,NaN,NaN,NaN,NaN,17,16,NaN,1C-WA-013-001,jlamb,01/11/2022 09:41:43,LAKE GEN,01/05/2022,01/12/2022,01/08/2022,NaN,NaN,NaN,NaN,S0,0.40,0.0
2,KMHT36AH1NU013136,3,HU,KMHT36AH1NU013136,[N]Normal Port Stock,HMA,[S2000]Stock,12/22/2021,2021122200307,[VL1]VELOSTER (JS),2022,[FN352FT5]MY22 VELOSTER N (JS),1,"[EXTRM]Extreme Transportation, Inc.",[CA323]NORM REEVES HYU SUPERSTORE CER,NaN,NaN,[CA323]NORM REEVES HYU SUPERSTORE CER,Dealer Car,CERRITOS,CA,[WE][KMA/HMA] Western,[MZH]Ultra Black,[NNB]Black,12/13/2021,01/11/2022,3679567,12/22/2021,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,97 JSCT01 JSPC01,"CT,PC",Light PIO,No,NaN,No cmnt hold,NaN,NaN,NaN,NaN,01/21/2022 15:25:36,17,16,NaN,04-06-081-001,jlamb,01/11/2022 09:50:41,LAKE GEN,01/05/2022,01/12/2022,01/08/2022,NaN,NaN,NaN,NaN,T3,0.00,0.0
3,KNDJ23AU4N7821344,4,HU,KNDJ23AU4N7821344,[N]Normal Port Stock,KMA,[S2000]Stock,11/19/2021,2021122300745,[SO1]SOUL (SK3),2022,[B2522]MY22 SOUL (SK3),10,[MPG]United Road,[NV012]Towbin Kia,NaN,01/18/2022,[NV012]Towbin Kia,Dealer Car,HENDERSON,NV,[WE][KMA/HMA] Western,[KDT]Gravity Grey,[WK]Black,12/11/2021,01/20/2022,P202111001,11/19/2021,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,SKCF02 SKCN02,"08,18,19,25,35,CF,CN",Light PIO,No,NaN,No cmnt hold,NaN,NaN,01/18/2022,01/20/2022,NaN,8,7,NaN,07-02-055-001,wfredrick,01/20/2022 13:19:48,DIGNITY,01/15/2022,01/21/2022,01/18/2022,NaN,NaN,NaN,NaN,J2,0.25,0.0
4,KNDJ23AU6N7822270,5,HU,KNDJ23AU6N7822270,[N]Normal Port Stock,KMA,[S2000]Stock,11/23/2021,2021122300745,[SO1]SOUL (SK3),2022,[B2522]MY22 SOUL (SK3),10,[MPG]United Road,[AZ049]PEORIA KIA,NaN,01/18/2022,[AZ049]PEORIA KIA,Dealer Car,PEORIA,AZ,[WE][KMA/HMA] Western,[M4A]Mars Orange,[GDM]Black/Gray,12/20/2021,01/20/2022,P202112001,11/23/2021,Y,[U]Unsold,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Truck,NaN,NaN,NaN,NaN,SKCF02,"08,18,19,25,35,CF",Throw-in PIO,No,NaN,No cmnt hold,NaN,NaN,01/18/2022,01/19/2022,NaN,8,7